In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools
using LinearAlgebra

## Funzione da ottimizzare

In [2]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:]) * abs.(EV))[1,:].nzind
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = face_int(tV, EV, FE[i, :])
		sV, sEV
        sV, sEV = skel_merge(sV, sEV, tmpV, tmpEV)
    end
    
    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

## Dipendenze della funzione

In [3]:
function face_int(V::Lar.Points, EV::Lar.ChainOp, face::Lar.Cell)
    vs = Lar.buildFV(EV, face)
    retV = Lar.Points(undef, 0, 3)
    visited_verts = []
    for i in 1:length(vs)
        o = V[vs[i],:]
        j = i < length(vs) ? i+1 : 1
        d = V[vs[j],:] - o

        err = 10e-8
        # err = 10e-4
        if !(-err < d[3] < err)

            alpha = -o[3] / d[3]

            if -err <= alpha <= 1+err
                p = o + alpha*d

                if -err < alpha < err || 1-err < alpha < 1+err
                    if !(Lar.vin(p, visited_verts))
                        push!(visited_verts, p)
                        retV = [retV; reshape(p, 1, 3)]
                    end
                else
                    retV = [retV; reshape(p, 1, 3)]
                end
            end
        end

    end

    vnum = size(retV, 1)


    if vnum == 1
        vnum = 0
        retV = Lar.Points(undef, 0, 3)
    end
    enum = (÷)(vnum, 2)
    retEV = spzeros(Int8, enum, vnum)

    for i in 1:enum
        retEV[i, 2*i-1:2*i] = [-1, 1]
    end

    retV, retEV
end

function submanifold_mapping(vs)
    u1 = vs[2,:] - vs[1,:]
    u2 = vs[3,:] - vs[1,:]
    u3 = cross(u1, u2)
    T = Matrix{Float64}(LinearAlgebra.I, 4, 4)
    T[4, 1:3] = - vs[1,:]
    M = Matrix{Float64}(LinearAlgebra.I, 4, 4)
    M[1:3, 1:3] = [u1 u2 u3]
    return T*M
end

function skel_merge(V1::Lar.Points, EV1::Lar.ChainOp, V2::Lar.Points, EV2::Lar.ChainOp)
    V = [V1; V2]
    EV = blockdiag(EV1,EV2)
    return V, EV
end
function skel_merge(V1::Lar.Points, EV1::Lar.ChainOp, FE1::Lar.ChainOp, V2::Lar.Points, EV2::Lar.ChainOp, FE2::Lar.ChainOp)
    FE = blockdiag(FE1,FE2)
    V, EV = skel_merge(V1, EV1, V2, EV2)
    return V, EV, FE
end


skel_merge (generic function with 2 methods)

## Dati in input

In [4]:
numThet = 60
b=[[(-4.0*numThet), (4.0*numThet)+20, ((4.0*numThet)/2), ((4.0*numThet)/2)],
[10.0, 10.0, -40.0, 5.0],
[0.0, 0.0, 0.0, -1.0]]
EV=[[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]
FV=[[1,2,3],[1,2,4],[1,3,4],[2,3,4]]
CV=[[1,2,3,4]]

for i=2:(numThet+1)
    push!(b[1],(1.0 + i*4))
    push!(b[2],(1.0))
    push!(b[3],(-0.5))

    push!(b[1],(4.0 + i*4))
    push!(b[2],(1.0))
    push!(b[3],(-0.5))

    push!(b[1],(2.5 + i*4))
    push!(b[2],(4.0))
    push!(b[3],(-0.5))

    push!(b[1],(2.5 + i*4))
    push!(b[2],(2.5))
    push!(b[3],(0.5))

    push!(EV,[1+4*(i-1),2+4*(i-1)])
    push!(EV,[1+4*(i-1),3+4*(i-1)])
    push!(EV,[1+4*(i-1),4+4*(i-1)])
    push!(EV,[2+4*(i-1),3+4*(i-1)])
    push!(EV,[2+4*(i-1),4+4*(i-1)])
    push!(EV,[3+4*(i-1),4+4*(i-1)])

    push!(FV,[1+4*(i-1),2+4*(i-1),3+4*(i-1)])
    push!(FV,[1+4*(i-1),2+4*(i-1),4+4*(i-1)])
    push!(FV,[1+4*(i-1),3+4*(i-1),4+4*(i-1)])
    push!(FV,[2+4*(i-1),3+4*(i-1),4+4*(i-1)])

    push!(CV,[1+4*(i-1),2+4*(i-1),3+4*(i-1),4+4*(i-1)])
end
V = permutedims(reshape(hcat(b...), (length(b[1]), length(b))))
sp_idx = Lar.spaceindex((V,FV))

copEV = Lar.coboundary_0(EV::Lar.Cells)
copFE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells)
V = convert(Array{Float64,2},V') 
sigma=1 #Faccia che interseca tutti i 240 tetraedri

1

## Benchmark vecchia funzione

In [11]:
@btime frag_face(V,copEV,copFE,sp_idx,sigma)[1]

  85.993 ms (747621 allocations: 41.79 MiB)


177×3 Array{Float64,2}:
 -240.0     10.0      0.0
  260.0     10.0      0.0
  120.0    -40.0      0.0
  237.935    2.11957  0.0
  238.894    2.46212  0.0
   11.25     1.75     0.0
    9.75     1.75     0.0
   10.5      3.25     0.0
   15.25     1.75     0.0
   13.75     1.75     0.0
   14.5      3.25     0.0
   19.25     1.75     0.0
   17.75     1.75     0.0
    ⋮                 
  221.75     1.75     0.0
  222.5      3.25     0.0
  227.25     1.75     0.0
  225.75     1.75     0.0
  226.5      3.25     0.0
  231.25     1.75     0.0
  229.75     1.75     0.0
  230.5      3.25     0.0
  235.25     1.75     0.0
  233.75     1.75     0.0
  234.5      3.25     0.0
  238.5      3.25     0.0

## Refactoring codice  
  
Analizzando il codice, ci siamo accorti che quando si applica la rototraslazione dei punti nel piano z=0 rispetto alla faccia sigma, essa viene applicata a tutti i punti V. (dell'intero modello!!!)  
  
Questo non è necessario, in quanto lo spaceindex ci dice già a priori quali facce intersecano con la faccia sigma, quindi è sufficiente applicare la rototraslazione ai soli punti di sigma e ai punti delle facce i contenute in spaceindex(sigma), non a tutti i punti V.

In [13]:
function face_int2(V::Lar.Points, EV::Lar.ChainOp, face::Lar.Cell)
    retV = Lar.Points(undef, 0, 3)
    visited_verts = []
    for i in 1:size(V,1)
        o = V[i,:]
        j = i < size(V,1) ? i+1 : 1
        d = V[j,:] - o
        err = 10e-8
        # err = 10e-4
        if !(-err < d[3] < err)

            alpha = -o[3] / d[3]

            if -err <= alpha <= 1+err
                p = o + alpha*d

                if -err < alpha < err || 1-err < alpha < 1+err
                    if !(Lar.vin(p, visited_verts))
                        push!(visited_verts, p)
                        retV = [retV; reshape(p, 1, 3)]
                    end
                else
                    retV = [retV; reshape(p, 1, 3)]
                end
            end
        end

    end

    vnum = size(retV, 1)


    if vnum == 1
        vnum = 0
        retV = Lar.Points(undef, 0, 3)
    end
    enum = (÷)(vnum, 2)
    retEV = spzeros(Int8, enum, vnum)

    for i in 1:enum
        retEV[i, 2*i-1:2*i] = [-1, 1]
    end

    retV, retEV
end

function frag_face2(V, EV, FE, sp_idx, sigma)
    vs_num = size(V, 1)
	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:]) * abs.(EV))[1,:].nzind
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = submanifold_mapping(sV)
    #APPLICO LA ROTO TRASLAZIONE AI SOLI PUNTI DELLA FACCIA SIGMA
    sV = ([sV ones(size(sV,1))]*M)[:, 1:3] 
    #Inizializzo vertici faccia i prima del for per evitare la distruzione 
    #e riallocazione a ogni iterazione...
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        faceivs = (abs.(FE[i:i,:]) * abs.(EV))[1,:].nzind
        faceiV = V[faceivs, :]
        #APPLICO LA ROTO TRASLAZIONE AI SOLI PUNTI DELLA FACCIA I CONTENUTA NELLO SPACEINDEX
        tV = ([faceiV ones(size(faceiV, 1))]*M)[:, 1:3]  
            tmpV, tmpEV = face_int2(tV, EV, FE[i, :])
            sV, sEV = skel_merge(sV, sEV, tmpV, tmpEV)
    end
    
    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    return nV, nEV, nFE
    
end

@btime frag_face2(V,copEV,copFE,sp_idx,sigma)[1]

  93.463 ms (732616 allocations: 44.02 MiB)


177×3 Array{Float64,2}:
 -240.0     10.0      0.0
  260.0     10.0      0.0
  120.0    -40.0      0.0
  237.935    2.11957  0.0
  238.894    2.46212  0.0
   11.25     1.75     0.0
    9.75     1.75     0.0
   10.5      3.25     0.0
   15.25     1.75     0.0
   13.75     1.75     0.0
   14.5      3.25     0.0
   19.25     1.75     0.0
   17.75     1.75     0.0
    ⋮                 
  221.75     1.75     0.0
  222.5      3.25     0.0
  227.25     1.75     0.0
  225.75     1.75     0.0
  226.5      3.25     0.0
  231.25     1.75     0.0
  229.75     1.75     0.0
  230.5      3.25     0.0
  235.25     1.75     0.0
  233.75     1.75     0.0
  234.5      3.25     0.0
  238.5      3.25     0.0

## Parallelizzazione cicli con i Threads  
  
Purtroppo non è stato possibile parallelizzare il ciclo sullo spaceindex di sigma, in quanto la chiamata sV, sEV = skel_merge(sV, sEV, tmpV, tmpEV)